In [1]:
# Define the Particle type
struct Particle
    x::Float64
    y::Float64
    state::Int  # 1 for free, 2 for bound
end


In [7]:
function simulate_states(k12, k21, state_changes=10,Δt = 0.1; max_steps=100000)
    
   
    p12 = k12 * Δt
    p21 = k21 * Δt
    
  
    states = Int[]
    current_state = 1
    push!(states, current_state)
    
    changes_count = 0
    steps = 0
    
    
    while changes_count < state_changes && steps < max_steps
        steps += 1
        r = rand()
        
       
        next_state = current_state
        if current_state == 1
            if r < p12
                next_state = 2
            end
        else
            if r < p21
                next_state = 1
            end
        end
        
        
        if next_state != current_state
            changes_count += 1
        end
        
        current_state = next_state
        push!(states, current_state)
    end
    
   
    
    
    return states, steps
end

function get_state_transitions(states::Vector{Int})
    transitions = []
    for i in 2:length(states)
        if states[i] != states[i - 1]
            t = (i - 1) 
            from_state = states[i - 1]
            to_state = states[i]
            push!(transitions, ( t, from_state, to_state))
        end
    end
    return transitions
end

get_state_transitions (generic function with 1 method)

In [2]:

# Modified constrained_diffusion function to use Particle type
function constrained_diffusion(; initial_p1 = nothing, D = 0.01, r = 0.2, box = 1.0, dt = 0.016, steps = 500)
    # Initialize particle 1 position
    if isnothing(initial_p1)
        p1_pos = rand(2) .* (box - 2r) .+ r
    else
        p1_pos = copy(initial_p1)
        for i in 1:2
            p1_pos[i] = clamp(p1_pos[i], r, box - r)
        end
    end
    
    # Initial angle and position for particle 2
    initial_angle = 2π * rand()
    p2_pos = p1_pos + r .* [cos(initial_angle), sin(initial_angle)]
    
    # Velocity for particle 1
    p1_vel = randn(2) .* sqrt(2D/dt)
    
    # Arrays to store particles
    p1_particles = Vector{Particle}(undef, steps)
    p2_particles = Vector{Particle}(undef, steps)
    
    # Initialize first position
    p1_particles[1] = Particle(p1_pos[1], p1_pos[2], 2)  # State 2 for bound
    p2_particles[1] = Particle(p2_pos[1], p2_pos[2], 2)  # State 2 for bound
    
    # Run simulation
    for frame in 2:steps
        p1_vel .= randn(2) .* sqrt(2D/dt)
        p1_pos .+= p1_vel .* dt
        
        for i in 1:2
            p1_pos[i] = clamp(p1_pos[i], 0, box)
        end
        
        new_angle = 2π * rand()
        p2_pos .= p1_pos + r .* [cos(new_angle), sin(new_angle)]
        
        for i in 1:2
            if p2_pos[i] <= 0 || p2_pos[i] >= box
                retry_count = 0
                while (p2_pos[i] <= 0 || p2_pos[i] >= box) && retry_count < 10
                    new_angle = 2π * rand()
                    p2_pos .= p1_pos + r .* [cos(new_angle), sin(new_angle)]
                    retry_count += 1
                end
                
                if p2_pos[i] <= 0 || p2_pos[i] >= box
                    if p1_pos[i] < box/2
                        p1_pos[i] = r + 0.05
                    else
                        p1_pos[i] = box - r - 0.05
                    end
                    p2_pos .= p1_pos + r .* [cos(new_angle), sin(new_angle)]
                end
            end
        end
        
        p1_particles[frame] = Particle(p1_pos[1], p1_pos[2], 2)  # State 2 for bound
        p2_particles[frame] = Particle(p2_pos[1], p2_pos[2], 2)  # State 2 for bound
    end
    
    return p1_particles, p2_particles
end


constrained_diffusion (generic function with 1 method)

In [3]:

# Modified constrained_diffusion function to use Particle type
function constrained_diffusion(; initial_p1 = nothing, D = 0.01, r = 0.2, box = 1.0, dt = 0.016, steps = 500)
    # Initialize particle 1 position
    if isnothing(initial_p1)
        p1_pos = rand(2) .* (box - 2r) .+ r
    else
        p1_pos = copy(initial_p1)
        for i in 1:2
            p1_pos[i] = clamp(p1_pos[i], r, box - r)
        end
    end
    
    # Initial angle and position for particle 2
    initial_angle = 2π * rand()
    p2_pos = p1_pos + r .* [cos(initial_angle), sin(initial_angle)]
    
    # Velocity for particle 1
    p1_vel = randn(2) .* sqrt(2D/dt)
    
    # Arrays to store particles
    p1_particles = Vector{Particle}(undef, steps)
    p2_particles = Vector{Particle}(undef, steps)
    
    # Initialize first position
    p1_particles[1] = Particle(p1_pos[1], p1_pos[2], 2)  # State 2 for bound
    p2_particles[1] = Particle(p2_pos[1], p2_pos[2], 2)  # State 2 for bound
    
    # Run simulation
    for frame in 2:steps
        p1_vel .= randn(2) .* sqrt(2D/dt)
        p1_pos .+= p1_vel .* dt
        
        for i in 1:2
            p1_pos[i] = clamp(p1_pos[i], 0, box)
        end
        
        new_angle = 2π * rand()
        p2_pos .= p1_pos + r .* [cos(new_angle), sin(new_angle)]
        
        for i in 1:2
            if p2_pos[i] <= 0 || p2_pos[i] >= box
                retry_count = 0
                while (p2_pos[i] <= 0 || p2_pos[i] >= box) && retry_count < 10
                    new_angle = 2π * rand()
                    p2_pos .= p1_pos + r .* [cos(new_angle), sin(new_angle)]
                    retry_count += 1
                end
                
                if p2_pos[i] <= 0 || p2_pos[i] >= box
                    if p1_pos[i] < box/2
                        p1_pos[i] = r + 0.05
                    else
                        p1_pos[i] = box - r - 0.05
                    end
                    p2_pos .= p1_pos + r .* [cos(new_angle), sin(new_angle)]
                end
            end
        end
        
        p1_particles[frame] = Particle(p1_pos[1], p1_pos[2], 2)  # State 2 for bound
        p2_particles[frame] = Particle(p2_pos[1], p2_pos[2], 2)  # State 2 for bound
    end
    
    return p1_particles, p2_particles
end

# Modified simulate_diffusion function to use Particle type
function simulate_diffusion(; initial_positions = nothing, D = 0.01, box = 1.0, dt = 0.016, steps = 2000)
    # Initialize positions
    if initial_positions isa Tuple && length(initial_positions) == 2
        p1_pos = copy(initial_positions[1])
        p2_pos = copy(initial_positions[2])
    else
        p1_pos = rand(2) .* box
        p2_pos = rand(2) .* box
    end
    
    # Initialize velocities
    p1_vel = randn(2) .* sqrt(2D/dt)
    p2_vel = randn(2) .* sqrt(2D/dt)
    
    # Arrays to store particles
    p1_particles = Vector{Particle}(undef, steps)
    p2_particles = Vector{Particle}(undef, steps)
    
    # Initialize first position
    p1_particles[1] = Particle(p1_pos[1], p1_pos[2], 1)  # State 1 for free
    p2_particles[1] = Particle(p2_pos[1], p2_pos[2], 1)  # State 1 for free
    
    # Run simulation
    for frame in 2:steps
        # Update velocities and positions
        p1_vel .= randn(2) .* sqrt(2D/dt)
        p2_vel .= randn(2) .* sqrt(2D/dt)
        
        p1_pos .+= p1_vel .* dt
        p2_pos .+= p2_vel .* dt
        
        # Enforce boundaries
        for i in 1:2
            p1_pos[i] = clamp(p1_pos[i], 0, box)
            p2_pos[i] = clamp(p2_pos[i], 0, box)
        end
        
        # Store particles
        p1_particles[frame] = Particle(p1_pos[1], p1_pos[2], 1)  # State 1 for free
        p2_particles[frame] = Particle(p2_pos[1], p2_pos[2], 1)  # State 1 for free
    end
    
    return p1_particles, p2_particles
end


simulate_diffusion (generic function with 1 method)

In [4]:
# Helper function to convert Particle arrays to position matrices
function particles_to_matrices(particles::Vector{Particle})
    n = length(particles)
    positions = zeros(2, n)
    
    for i in 1:n
        positions[1, i] = particles[i].x
        positions[2, i] = particles[i].y
    end
    
    return positions
end

particles_to_matrices (generic function with 1 method)

In [5]:



# Modified simulation function to use Particle type
function simulation(k_in, k12_off, changes)
    k12 = k_in
    k21 = k12_off
    states, steps = simulate_states(k12, k21, changes)
    time_in_state = time_sequence_with_split(states)
    
    current_time = 0
    p1_particles = Particle[]
    p2_particles = Particle[]
    
    for i in 1:length(time_in_state)
        section_time_stamps = time_stamps(time_in_state[i])
        modes = modes_selection(section_time_stamps)
        
        if i == 1
            if modes == :full || modes == :middle || modes == :left_half || modes == :right_half
                # Get raw positions first
                p1_pos, p2_pos = segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
                
                # Convert to particles with appropriate states
                temp_p1 = Vector{Particle}(undef, size(p1_pos, 2))
                temp_p2 = Vector{Particle}(undef, size(p2_pos, 2))
                
                for j in 1:size(p1_pos, 2)
                    # Determine state based on mode and time segment
                    state_val = modes == :middle || 
                               (modes == :right_half && j <= section_time_stamps[1]) || 
                               (modes == :left_half && j > section_time_stamps[1]) || 
                               (modes == :full && j > section_time_stamps[1] && j <= section_time_stamps[1] + section_time_stamps[2]) ? 2 : 1
                    
                    temp_p1[j] = Particle(p1_pos[1, j], p1_pos[2, j], state_val)
                    temp_p2[j] = Particle(p2_pos[1, j], p2_pos[2, j], state_val)
                end
                
                p1_particles = temp_p1
                p2_particles = temp_p2
            end
        else
            if modes == :full || modes == :middle || modes == :left_half || modes == :right_half
                # Get raw positions first
                p1_pos_temp, p2_pos_temp = segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
                
                # Apply shift
                shift_x = -p1_pos_temp[1, 1] + p1_particles[end].x
                shift_y = -p1_pos_temp[2, 1] + p1_particles[end].y
                
                p1_pos = hcat(zeros(2, 1), p1_pos_temp)
                p2_pos = hcat(zeros(2, 1), p2_pos_temp)
                
                p1_pos[1, 2:end] .+= shift_x
                p1_pos[2, 2:end] .+= shift_y
                p2_pos[1, 2:end] .+= shift_x
                p2_pos[2, 2:end] .+= shift_y
                
                # Apply path correction to the first segment if needed
                segment_size = section_time_stamps[1]
                if segment_size > 0
                    segment_view1 = @view p1_pos[:, 2:segment_size+1]
                    segment_view2 = @view p2_pos[:, 2:segment_size+1]
                    
                    path_correction!(segment_view2, [p2_particles[end].x, p2_particles[end].y])
                end
                
                # Convert to particles with appropriate states
                temp_p1 = Vector{Particle}(undef, size(p1_pos, 2) - 1)
                temp_p2 = Vector{Particle}(undef, size(p2_pos, 2) - 1)
                
                for j in 2:size(p1_pos, 2)
                    # Determine state based on mode and time segment
                    state_val = modes == :middle || 
                               (modes == :right_half && j-1 <= section_time_stamps[1]) || 
                               (modes == :left_half && j-1 > section_time_stamps[1]) || 
                               (modes == :full && j-1 > section_time_stamps[1] && j-1 <= section_time_stamps[1] + section_time_stamps[2]) ? 2 : 1
                    
                    temp_p1[j-1] = Particle(p1_pos[1, j], p1_pos[2, j], state_val)
                    temp_p2[j-1] = Particle(p2_pos[1, j], p2_pos[2, j], state_val)
                end
                
                # Append to particle arrays
                append!(p1_particles, temp_p1)
                append!(p2_particles, temp_p2)
            end
        end
        
        current_time = length(p1_particles)
    end
    
    return p1_particles, p2_particles
end

simulation (generic function with 1 method)

In [9]:

function time_sequence(states::Vector{Int})
    if isempty(states)
        return []
    end
    
    state_time_sequence = []
    current_state = states[1]
    current_count = 1
    
    # Iterate through states starting from the second element
    for i in 2:length(states)
        if states[i] == current_state
            # Same state, increment count
            current_count += 1
        else
            # State changed, record the previous state and its duration
            push!(state_time_sequence, current_state => current_count)
            # Start counting the new state
            current_state = states[i]
            current_count = 1
        end
    end
    
    # Don't forget to add the last state
    push!(state_time_sequence, current_state => current_count)
    
    return state_time_sequence
end
function divide_into_sections(transitions::Vector, section_size::Int=3)
    sections = []
    current_section = []
    
    for (i, transition) in enumerate(transitions)
        push!(current_section, transition)
        
        # When we reach section_size or the end of the sequence, add the section
        if i % section_size == 0 || i == length(transitions)
            push!(sections, current_section)
            current_section = []
        end
    end
    
    return sections
end

function time_sequence_with_split(states::Vector{Int})
    if isempty(states)
        return []
    end
    
    # Calculate raw state durations
    durations = []
    current_state = states[1]
    current_count = 1
    
    for i in 2:length(states)
        if states[i] == current_state
            current_count += 1
        else
            push!(durations, current_state => current_count)
            current_state = states[i]
            current_count = 1
        end
    end
    push!(durations, current_state => current_count)
    
    # Create sections
    sections = []
    current_section = []
    
    # Process the first element normally (won't be split)
    if !isempty(durations)
        push!(current_section, durations[1])
    end
    
    # Process the rest of the durations
    i = 2
    while i <= length(durations)
        state, duration = durations[i].first, durations[i].second
        
        if state == 1 && duration > 1
            # Split state 1 in half
            half = div(duration, 2)
            
            # Add first half to current section
            push!(current_section, 1 => half)
            
            # Check if section is full
            if length(current_section) == 3
                push!(sections, current_section)
                current_section = []
            end
            
            # Start new section with second half
            push!(current_section, 1 => (duration - half))
        else
            # Regular state, add normally
            push!(current_section, state => duration)
        end
        
        # If section is full, add it and start a new one
        if length(current_section) == 3
            push!(sections, current_section)
            current_section = []
        end
        
        i += 1
    end
    
    # Add any remaining elements
    if !isempty(current_section)
        push!(sections, current_section)
    end
    
    return sections
end

time_sequence_with_split (generic function with 1 method)

In [11]:
function get_state_transitions(states::Vector{Int})
    transitions = []
    for i in 2:length(states)
        if states[i] != states[i - 1]
            t = (i - 1) 
            from_state = states[i - 1]
            to_state = states[i]
            push!(transitions, ( t, from_state, to_state))
        end
    end
    return transitions
end

get_state_transitions (generic function with 1 method)

In [13]:
function time_stamps(time_state)

    segments = []
    for i in 1:length(time_state)
        push!(segments, time_state[i].second)
    end


    return segments
end 

time_stamps (generic function with 1 method)

In [15]:
function modes_selection(section::Vector )

    size_section = length(section)
    if size_section == 3
        return :full
    elseif size_section == 2
        if section[1].first == 1 && section[2].first == 2
            return :left_half
        elseif section[1].first == 2 && section[2].first == 1
            return :right_half
        else
            error("Invalid section combination")
        end
    elseif size_section == 1
        return :middle
    else
        error("Invalid number of sections")
    end

end 

modes_selection (generic function with 1 method)

In [17]:
function segments(time_division::Vector, mode::Symbol;  D = 0.01, r = 0.2, box = 1.0, dt = 0.016)

    if mode == :full && length(time_division) == 3 # all the sections 
    
            p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],    D = D, r = r, box = box, dt = dt)
    
        init_post_r = ([p1_middle[1,end], p1_middle[2,end]],[p2_middle[1,end], p2_middle[2,end]])
        p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[3],  D = D, box = box, dt = dt)
    
        init_post_l = ([p1_middle[1,1], p1_middle[2,1]], [p2_middle[1,1], p2_middle[2,1]])
        p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l, steps = time_division[1], D = D, box = box, dt = dt)
        p1_left_reversed = reverse_columns_preserve_size(p1_left)
        p2_left_reversed = reverse_columns_preserve_size(p2_left)
    
        p1 = hcat(p1_left_reversed, p1_middle, p1_right)
        p2 = hcat(p2_left_reversed, p2_middle, p2_right)
    
        return p1, p2
    
    elseif mode == :right_half && length(time_division) == 2 # middle and right half
    
        p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)
    
        init_post_r = ([p1_middle[1,end], p1_middle[2,end]],  [p2_middle[1,end], p2_middle[2,end]])
        p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[2],  D = D, box = box, dt = dt)
    
        p1 = hcat(p1_middle, p1_right)
        p2 = hcat(p2_middle, p2_right)
    
        return p1, p2
    
    elseif mode == :middle && length(time_division) == 1 # only middle half 
        p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)
        p1 = hcat(p1_middle)
        p2 = hcat(p2_middle)
    
        return p1, p2
    
    elseif mode == :left_half && length(time_division) == 2 # middle and left half
        p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],   D = D, r = r, box = box, dt = dt)
    
        init_post_l = ([p1_middle[1,1], p1_middle[2,1]],  [p2_middle[1,1], p2_middle[2,1]])
        p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l,  steps = time_division[1], D = D, box = box, dt = dt)
        p1_left_reversed = reverse_columns_preserve_size(p1_left)
        p2_left_reversed = reverse_columns_preserve_size(p2_left)
    
        p1 = hcat(p1_left_reversed, p1_middle)
        p2 = hcat(p2_left_reversed, p2_middle)
        return p1, p2
    
    else
        error("Invalid mode or time division")
    end 
    end

segments (generic function with 1 method)

In [18]:
k12 = 0.5       # Rate of binding
k21 = 0.15      # Rate of unbinding
changes = 6     # Number of state changes

# Run simulation
p1, p2 = simulation(k12, k21, changes)

MethodError: MethodError: Cannot `convert` an object of type Particle to an object of type Float64
The function `convert` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:126
  convert(::Type{T}, !Matched::AbstractChar) where T<:Number
   @ Base char.jl:185
  ...
